# 列画像と文字列の可視化

このノートブックでは、検出された列の画像と対応する文字列を表示します。

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
def unicode_ids_to_string(unicode_ids: list) -> str:
    """Unicode IDのリストを文字列に変換

    Args:
        unicode_ids (list): Unicode IDのリスト（文字列表現）

    Returns:
        str: 変換された文字列
    """
    # 文字列表現からリストに変換
    if isinstance(unicode_ids, str):
        unicode_ids = eval(unicode_ids)
    
    # Unicode IDを文字に変換
    return ''.join([chr(int(uid, 16)) for uid in unicode_ids])

In [ ]:
def display_column_info(
    column_image_path: str,
    unicode_ids: list,
    figsize: tuple = (5, 10)
) -> None:
    """列画像と文字列を表示

    Args:
        column_image_path (str): 列画像のパス
        unicode_ids (list): Unicode IDのリスト
        figsize (tuple, optional): 図のサイズ. Defaults to (5, 10).
    """
    # 画像の読み込みと表示
    image = cv2.imread(column_image_path)
    if image is None:
        print(f"Error: Could not load image from {column_image_path}")
        return
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis('off')
    
    # 文字列の表示
    text = unicode_ids_to_string(unicode_ids)
    plt.title(f'Text: {text}', fontsize=12, pad=20)
    
    plt.show()

## 列情報の読み込みと表示

In [ ]:
# 列情報の読み込み
column_info_df = pd.read_csv('data/processed/column_info.csv')
print(f"Loaded {len(column_info_df)} columns")

In [ ]:
# 最初の10列を表示
for i in range(min(10, len(column_info_df))):
    row = column_info_df.iloc[i]
    print(f"\nColumn {i+1}:")
    display_column_info(row['column_image'], row['unicode_ids'])

## 特定の画像の列を表示

In [ ]:
# 元画像のパスで列を検索
def display_columns_for_image(original_image_path: str) -> None:
    """指定された元画像から抽出された列を表示

    Args:
        original_image_path (str): 元画像のパス
    """
    # 該当する列を抽出
    columns = column_info_df[column_info_df['original_image'] == original_image_path]
    
    if len(columns) == 0:
        print(f"No columns found for image: {original_image_path}")
        return
    
    print(f"Found {len(columns)} columns for image: {original_image_path}")
    
    # 各列を表示
    for i, (_, row) in enumerate(columns.iterrows(), 1):
        print(f"\nColumn {i}:")
        display_column_info(row['column_image'], row['unicode_ids'])

In [ ]:
# 利用可能な元画像のリストを表示
original_images = column_info_df['original_image'].unique()
print("Available original images:")
for i, image_path in enumerate(original_images, 1):
    print(f"{i}. {image_path}")

In [ ]:
# 最初の画像の列を表示
if len(original_images) > 0:
    display_columns_for_image(original_images[0])

## 文字数による列のフィルタリング

In [ ]:
# 文字数を計算
column_info_df['char_count'] = column_info_df['unicode_ids'].apply(
    lambda x: len(eval(x)) if isinstance(x, str) else len(x)
)

# 文字数の分布を表示
plt.figure(figsize=(10, 5))
column_info_df['char_count'].hist(bins=50)
plt.title('Distribution of Characters per Column')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')
plt.show()

# 基本統計量を表示
print("\nCharacter count statistics:")
print(column_info_df['char_count'].describe())

In [ ]:
# 特定の文字数範囲の列を表示
def display_columns_by_char_count(min_chars: int, max_chars: int, n_samples: int = 5) -> None:
    """指定された文字数範囲の列をランダムに表示

    Args:
        min_chars (int): 最小文字数
        max_chars (int): 最大文字数
        n_samples (int, optional): 表示するサンプル数. Defaults to 5.
    """
    # 条件に合う列を抽出
    filtered_columns = column_info_df[
        (column_info_df['char_count'] >= min_chars) &
        (column_info_df['char_count'] <= max_chars)
    ]
    
    if len(filtered_columns) == 0:
        print(f"No columns found with {min_chars}-{max_chars} characters")
        return
    
    # ランダムにサンプリング
    samples = filtered_columns.sample(n=min(n_samples, len(filtered_columns)))
    
    print(f"Displaying {len(samples)} random samples with {min_chars}-{max_chars} characters")
    
    # 各サンプルを表示
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        print(f"\nSample {i} ({row['char_count']} characters):")
        display_column_info(row['column_image'], row['unicode_ids'])

In [ ]:
# 10-20文字の列を5つ表示
display_columns_by_char_count(10, 20, 5)